In [1]:
import tensorflow as tf
import jpeg_related_functions as jrf
import multilayerCSC_ADMM as mlcsc
import numpy as np
import pickle as pkl
import datetime

In [2]:
rho = 1.
alpha_init = 1.5
mu_init = 1.
b_init = 0.
lraParam = {'n_components': 3}
cmplxdtype = tf.complex128 # This should really be elsewhere.
batch_size = 8
noe_per_save = 1
num_of_saves = 2
step_size = 0.1

In [3]:
experimentpath = 'data/experiment/simpleTest/experiment1/'
def checkpointfilename(ii):
    return 'checkpoint_epoch' + str(ii) + '.ckpt'
modelfilename = 'initial_model.ckpt'
fid = open(experimentpath + 'problem_param.pckl','rb')
problem_param = pkl.load(fid)
fid.close()
data_param = problem_param['data_param']
targetSz = data_param['target_size']
qY = data_param['qY']
qUV = data_param['qUV']
strides = problem_param['stride']
fltrSz = problem_param['fltrSz']
real_dtype = data_param['dtype']
noi = problem_param['noi']
noL = problem_param['noL']
noc = problem_param['noc']
datapath = problem_param['datapath']
trainfile = problem_param['trainfile']
padding = data_param['padding']


In [4]:
cropAndMerge = mlcsc.CropPadObject(targetSz,strides,[np.asarray(ks) for ks in fltrSz],real_dtype)
paddingTuple = cropAndMerge.paddingTuple
fftSz = cropAndMerge.get_fft_size(targetSz,strides)
paddingDiff = ((padding[0][0] - paddingTuple[0][0],padding[0][1] - paddingTuple[0][1]),(padding[1][0] - paddingTuple[1][0],padding[1][1] - paddingTuple[1][1]))
assert(paddingDiff[0][0] >= 0)
assert(paddingDiff[0][1] >= 0)
assert(paddingDiff[1][0] >= 0)
assert(paddingDiff[1][1] >= 0)
print(paddingDiff)


((4, 4), (4, 4))


In [5]:
startr = paddingDiff[0][0]
startc = paddingDiff[1][0]
endr = targetSz[0] + padding[0][0] + padding[0][1] - paddingDiff[0][1]
endc = targetSz[1] + padding[1][0] + padding[1][1] - paddingDiff[1][1]
example_structure = {'highpass': tf.io.FixedLenFeature([], tf.string), 'lowpass': tf.io.FixedLenFeature([], tf.string), 'compressed': tf.io.FixedLenFeature([], tf.string),'raw': tf.io.FixedLenFeature([], tf.string)}

def restore_double(x):
    return tf.io.parse_tensor(x,real_dtype)

def _parse_image_function(example_proto):
    x = tf.io.parse_single_example(example_proto, example_structure)
    highpass = restore_double(x['highpass'])
    lowpass = restore_double(x['lowpass'])
    return ((highpass[slice(startr,endr),slice(startc,endc),slice(None)],lowpass[slice(startr,endr),slice(startc,endc),slice(None)],restore_double(x['compressed'])),restore_double(x['raw']))

raw_dataset = tf.data.TFRecordDataset([datapath + trainfile])
dataset = raw_dataset.map(_parse_image_function)
dataset_batch = dataset.batch(batch_size)


In [6]:
for (x,y) in dataset_batch:
    print(x[0].shape)
    print(x[1].shape)
    print(x[2].shape)
    print(y.shape)
    print(tf.reduce_max(tf.abs(cropAndMerge.crop(x[0]) + cropAndMerge.crop(x[1]) - x[2])))
    croppedHighpass = x[0][slice(None),slice(paddingTuple[0][0], paddingTuple[0][0] + targetSz[0]),slice(paddingTuple[1][0],paddingTuple[1][0] + targetSz[1]),slice(None)]
    croppedLowpass = x[1][slice(None),slice(paddingTuple[0][0],paddingTuple[0][0] + targetSz[0]),slice(paddingTuple[1][0],paddingTuple[1][0] + targetSz[1]),slice(None)]
    print(croppedHighpass.shape)
    print(croppedLowpass.shape)
    print(x[2].shape)
    print(tf.reduce_max(tf.abs(croppedHighpass + croppedLowpass - x[2])))
    print(tf.reduce_max(x[2]))
    print(tf.reduce_min(x[2]))
    break

(8, 66, 66, 3)
(8, 66, 66, 3)
(8, 48, 48, 3)
(8, 48, 48, 3)
tf.Tensor(2.7755575615628914e-17, shape=(), dtype=float64)
(8, 48, 48, 3)
(8, 48, 48, 3)
(8, 48, 48, 3)
tf.Tensor(2.7755575615628914e-17, shape=(), dtype=float64)
tf.Tensor(1.2229405880737936, shape=(), dtype=float64)
tf.Tensor(-0.1560929926334258, shape=(), dtype=float64)


In [7]:
CSC = mlcsc.MultiLayerCSC(rho,alpha_init,mu_init,b_init,qY,qUV,cropAndMerge,fftSz,strides,problem_param['D'],lraParam,noi,noL,cmplxdtype)

In [8]:
highpassShape = (targetSz[0] + paddingTuple[0][0] + paddingTuple[0][1],targetSz[1] + paddingTuple[1][0] + paddingTuple[1][1],noc)
highpass = tf.keras.Input(shape=highpassShape,dtype=real_dtype)
lowpass = tf.keras.Input(shape = highpassShape,dtype = real_dtype)
compressed = tf.keras.Input(shape = (targetSz[0],targetSz[1],noc),dtype= real_dtype)
inputs = (highpass,lowpass,compressed)

reconstruction,itstats = CSC(inputs)
import post_process_grad as ppg
model = ppg.Model_PostProcess(inputs,reconstruction)

In [9]:
for (x,y) in dataset:
    x_input = x
    x_output = y
    break

recon = CSC(x_input)
print(tf.reduce_max(recon))
print(tf.reduce_min(recon))

InvalidArgumentError: -input rank(-3) <= split_dim < input rank (3), but got 3 [Op:Split] name: split

In [9]:
z_new = tf.where(highpass < 0,0.,highpass)
z_new = tf.where(z_new > 1.0,1.0,z_new)
jpegImage = Image.fromarray((255.*np.asarray(tf.reshape(z_new,z_new.shape[1:]))).astype('uint8'),'RGB')
jpegImage.show()
jpegImage2 = Image.fromarray((-255.*np.asarray(tf.reshape(z_new,z_new.shape[1:]))).astype('uint8'),'RGB')
jpegImage2.show()

In [10]:
rho = 1.
alpha_init = 1.5
mu_init = 1.
fltrSz = ((3,3),(3,3))
cropAndMerge = mlcsc.CropPadObject(np.asarray([24,24]),np.asarray([1,1]),[np.asarray(ks) for ks in fltrSz],highpass.dtype)
noc = (3,8)
nof = (8,16)
lraParam = {'n_components': 3}
noi = 3
noL = 2
cmplxdtype = tf.complex128
CSC = mlcsc.MultiLayerCSC(rho,alpha_init,mu_init,qY,qUV,cropAndMerge,fltrSz,(fftSz,fftSz),noc,nof,lraParam,noi,noL,cmplxdtype)

In [11]:
print(cropAndMerge.paddingTuple)

((3, 4), (3, 4))


In [11]:
cropped = cropAndMerge.crop(highpass)

In [12]:
print(cropped.shape)
print(cropped.dtype)

(1, 25, 25, 3)
<dtype: 'float64'>


In [13]:
x, itstats = CSC((lowpass,highpass,jpeg_image))

InvalidArgumentError: Incompatible shapes: [1,25,25,3] vs. [1,32,32,3] [Op:AddV2]

In [11]:
s = CSC.preprocess((lowpass,highpass,jpeg_image))
y,u,By,negC,itstats = CSC.init_vars(s)
x,Ax = CSC.xstep(y,u,By,negC)

In [220]:
Ax_relaxed = CSC.relax(Ax,By,negC)

In [221]:
beforeyupdate = CSC.evaluateRelaxedLagrangian(x,y,u,Ax_relaxed,By,negC)
#beforeyupdate = CSC.evaluateLagrangian(x,y,u,By,negC)
y,By = CSC.ystep(x,u,Ax_relaxed,negC)
#afteryupdate = CSC.evaluateLagrangian(x,y,u,By,negC)
afteryupdate = CSC.evaluateRelaxedLagrangian(x,y,u,Ax_relaxed,By,negC)

print('y-step improvement: ',beforeyupdate - afteryupdate)

y-step improvement:  tf.Tensor(2.8768026449496915, shape=(), dtype=float64)


In [222]:
u = CSC.ustep(u,Ax_relaxed,By,negC)

In [223]:
beforexupdate = CSC.evaluateLagrangian(x,y,u,By,negC)
x,Ax = CSC.xstep(y,u,By,negC)
afterxupdate = CSC.evaluateLagrangian(x,y,u,By,negC)

print('x-step improvement: ',beforexupdate - afterxupdate)

x-step improvement:  tf.Tensor(2.504578839214304, shape=(), dtype=float64)


In [84]:
print(x[0].dtype)

<dtype: 'complex128'>


In [15]:
print(x[0].dtype.real_dtype)

<dtype: 'float64'>


In [284]:
print(tf.reduce_max(tf.reshape(CSC.dictObj[0].R,(1,1,1,8,1))*CSC.IFFT[0](x[0])))

tf.Tensor(0.3018569557058902, shape=(), dtype=float64)


In [285]:
print(tf.reduce_min(CSC.IFFT[0](x[0])))

tf.Tensor(-0.018461204382282353, shape=(), dtype=float64)


In [286]:
print(tf.reduce_max(CSC.IFFT[0](y[0])))

tf.Tensor(1.2742018990043937, shape=(), dtype=float64)


In [287]:
print(tf.reduce_max(highpass))

tf.Tensor(0.2395837879066285, shape=(), dtype=float64)


In [348]:
print(tf.reduce_min(CSC.IFFT[0](y[0])))

tf.Tensor(-9.481495837540637, shape=(), dtype=float64)


In [351]:
print(np.amax(highpass))

0.30236045126053435


In [350]:
print([]*4)

[]


In [12]:
print([[],]*4)

[[], [], [], []]


In [13]:
tf.dtypes('complex128')

TypeError: 'module' object is not callable